In [50]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', None)

In [57]:
# Load data
portfolio = pd.read_csv('positions.csv')
fx_rates = pd.read_csv('fx.csv') 
fx_rates.drop(columns=['Unnamed: 2'], inplace=True)  

portfolio.head()
fx_rates

,currency,to_USD
0,AUD,0.774395
1,BRL,0.304710
2,CAD,0.747211
3,CHF,1.046150
4,CNY,0.158037
5,EUR,1.107643
6,GBP,1.344329
7,HKD,0.130361
8,JPY,0.009876
9,USD,1.000000


# Data Cleaning

In [58]:
# Data cleaning
# rows with any missing values
display(portfolio[portfolio.isnull().any(axis=1)])

# duplicates
display(portfolio[portfolio.duplicated(subset=['stock_id'], keep=False)])

#make sure numeric columns are properly typed
numeric_cols = ['posn_shares', 'cost_basis_local', 'market_price_local', 
                'beta', 'avg_daily_volume']
portfolio[numeric_cols] = portfolio[numeric_cols].apply(pd.to_numeric, errors='coerce')

# Check for scientific notation issues (your ADV has "1.36E+08")
# This should parse fine, but verify

# Side should be 'LONG' or 'SHORT' only
assert portfolio['side'].isin(['LONG', 'SHORT']).all()


,stock_id,name,ticker,country,currency,sector,industry,sub_industry,beta,avg_daily_volume,side,posn_shares,cost_basis_local,market_price_local
2296,2297,equity_2297,MOM,AUS,NaN,Industrials,Transportation,Marine,0.589532,36086448,LONG,3864,135.73,137.41
2305,2306,equity_2306,ONW,ITA,NaN,Consumer Discretionary,Media,Movies & Entertainment,1.310975,81798,SHORT,-3242,158.89,168.70


,stock_id,name,ticker,country,currency,sector,industry,sub_industry,beta,avg_daily_volume,side,posn_shares,cost_basis_local,market_price_local


In [59]:
# Impute missing currency values
portfolio.loc[portfolio['stock_id'] == 2297, 'currency'] = 'AUD'
portfolio.loc[portfolio['stock_id'] == 2306, 'currency'] = 'EUR'

# Verify
portfolio[portfolio['stock_id'].isin([2297, 2306])]


,stock_id,name,ticker,country,currency,sector,industry,sub_industry,beta,avg_daily_volume,side,posn_shares,cost_basis_local,market_price_local
2296,2297,equity_2297,MOM,AUS,AUD,Industrials,Transportation,Marine,0.589532,36086448,LONG,3864,135.73,137.41
2305,2306,equity_2306,ONW,ITA,EUR,Consumer Discretionary,Media,Movies & Entertainment,1.310975,81798,SHORT,-3242,158.89,168.70


# Conversion to USD

In [60]:
# Convert all positions to a common currency (USD)
# Calculate position values in local, then convert
portfolio['position_value_local'] = portfolio['posn_shares'] * portfolio['market_price_local']
portfolio['cost_value_local'] = portfolio['posn_shares'] * portfolio['cost_basis_local']
portfolio['unrealized_pnl_local'] = (portfolio['position_value_local'] - portfolio['cost_value_local']) * np.where(portfolio['side'] == 'LONG', 1, -1)

# Merge with FX rates
portfolio = portfolio.merge(fx_rates, on='currency', how='left')

# # convert to USD
portfolio['position_value_usd'] = portfolio['position_value_local'] * portfolio['to_USD']
portfolio['cost_value_usd'] = portfolio['cost_value_local'] * portfolio['to_USD']
portfolio['unrealized_pnl_usd'] = portfolio['unrealized_pnl_local'] * portfolio['to_USD']

portfolio.head()

,stock_id,name,ticker,country,currency,sector,industry,sub_industry,beta,avg_daily_volume,side,posn_shares,cost_basis_local,market_price_local,position_value_local,cost_value_local,unrealized_pnl_local,to_USD,position_value_usd,cost_value_usd,unrealized_pnl_usd
0,1,equity_1,AGI,DEU,EUR,Information Technology,Software & Services,Systems Software,1.735624,10043,SHORT,-4810,145.19,147.06,-707358.60,-698363.90,8994.70,1.107643,-7.835008e+05,-7.735379e+05,9962.916501
1,2,equity_2,PWH,RUS,USD,Consumer Staples,Food Beverage & Tobacco,Meat Poultry & Fish,0.675354,640830,LONG,38342,112.48,100.28,3844935.76,4312708.16,-467772.40,1.000000,3.844936e+06,4.312708e+06,-467772.400000
2,3,equity_3,NDE,DEU,USD,Industrials,Capital Goods,Heavy Electrical Equipment,0.699394,36094,SHORT,-19881,43.36,41.56,-826254.36,-862040.16,-35785.80,1.000000,-8.262544e+05,-8.620402e+05,-35785.800000
3,4,equity_4,ABB,USA,USD,Financials,Insurance,Reinsurance,2.215079,135786553,SHORT,-100194,24.79,26.38,-2643117.72,-2483809.26,159308.46,1.000000,-2.643118e+06,-2.483809e+06,159308.460000
4,5,equity_5,WRF,RUS,USD,Utilities,Utilities,Electric Utilities,1.259133,26094,LONG,3963,149.90,149.53,592587.39,594053.70,-1466.31,1.000000,5.925874e+05,5.940537e+05,-1466.310000


# Analysis

In [67]:
portfolio_sorted = portfolio.sort_values(by='position_value_usd', key=abs, ascending=False)
portfolio_sorted.head(10)

,stock_id,name,ticker,country,currency,sector,industry,sub_industry,beta,avg_daily_volume,side,posn_shares,cost_basis_local,market_price_local,position_value_local,cost_value_local,unrealized_pnl_local,to_USD,position_value_usd,cost_value_usd,unrealized_pnl_usd
2969,2970,equity_2970,ZXT,JPN,JPY,Health Care,Pharmaceuticals & Biotechnology,Biotechnology,0.808189,3849021,SHORT,-102182,18579.06,19463.31,-1.988800e+09,-1.898446e+09,90354433.50,0.009876,-1.964231e+07,-1.874993e+07,8.923822e+05
2708,2709,equity_2709,ZYD,BRA,BRL,Health Care,Pharmaceuticals & Biotechnology,Life Sciences Tools & Services,0.522965,2976122332,SHORT,-336407,197.40,181.65,-6.110833e+07,-6.640674e+07,-5298410.25,0.304710,-1.862032e+07,-2.023480e+07,-1.614479e+06
1974,1975,equity_1975,GVP,GBR,USD,Consumer Staples,Household & Personal Products,Household Products,0.554347,1223740291,LONG,136001,110.03,112.14,1.525115e+07,1.496419e+07,286962.11,1.000000,1.525115e+07,1.496419e+07,2.869621e+05
1185,1186,equity_1186,TNX,FIN,EUR,Consumer Staples,Household & Personal Products,Household Products,0.878405,385235496,LONG,79753,147.75,160.13,1.277085e+07,1.178351e+07,987342.14,1.107643,1.414554e+07,1.305192e+07,1.093623e+06
657,658,equity_658,PJZ,GBR,GBP,Health Care,Pharmaceuticals & Biotechnology,Biotechnology,1.443002,135396,LONG,93312,107.69,110.69,1.032871e+07,1.004877e+07,279936.00,1.344329,1.388518e+07,1.350886e+07,3.763262e+05
2755,2756,equity_2756,VSU,BEL,EUR,Health Care,Pharmaceuticals & Biotechnology,Life Sciences Tools & Services,1.892090,1878600640,SHORT,-199985,56.74,61.09,-1.221708e+07,-1.134715e+07,869934.75,1.107643,-1.353217e+07,-1.256859e+07,9.635771e+05
2929,2930,equity_2930,DFB,BRA,BRL,Health Care,Pharmaceuticals & Biotechnology,Pharmaceuticals,3.108001,56968,LONG,111590,358.78,381.37,4.255708e+07,4.003626e+07,2520818.10,0.304710,1.296757e+07,1.219945e+07,7.681186e+05
1219,1220,equity_1220,YWC,GRC,EUR,Consumer Staples,Household & Personal Products,Household Products,0.742857,75944185,SHORT,-274338,38.54,39.37,-1.080069e+07,-1.057299e+07,227700.54,1.107643,-1.196331e+07,-1.171109e+07,2.522109e+05
1630,1631,equity_1631,PRU,ESP,EUR,Health Care,Pharmaceuticals & Biotechnology,Biotechnology,0.835935,143951798,LONG,89097,119.44,120.06,1.069699e+07,1.064175e+07,55240.14,1.107643,1.184844e+07,1.178726e+07,6.118635e+04
2609,2610,equity_2610,AOG,USA,USD,Health Care,Pharmaceuticals & Biotechnology,Biotechnology,1.574179,1583674483,SHORT,-166051,73.83,70.14,-1.164682e+07,-1.225955e+07,-612728.19,1.000000,-1.164682e+07,-1.225955e+07,-6.127282e+05
